In [7]:
# 数値計算に使うライブラリ
import numpy as np
import pandas as pd
import scipy as sp
# 統計モデルを推定するライブラリ
import statsmodels.formula.api as smf
import statsmodels.api as sm
# 多層パーセプトロンを適用
from sklearn.neural_network import MLPClassifier
# サンプルデータの読み込み
from sklearn.datasets import load_iris
# テストデータと訓練データに分ける
from sklearn.model_selection import train_test_split
# データの標準化を行う
from sklearn.preprocessing import StandardScaler
# 表示桁数の指定
%precision 3

# 4-11
iris = load_iris()
iris.feature_names
iris.target_names
# 説明変数をsepal(ガク)だけにする
X = iris.data[50:150, 0:2]
# アヤメを2種類だけにする
y = iris.target[50:150]
'''
print("説明変数の行数・列数:", X.shape)
print("応答変数の行数・列数:", y.shape)
'''
# データをテストデータと訓練データに分ける
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state = 2)

'''
print("説明変数の行数・列数:", X_train.shape)
print("応答変数の行数・列数:", y_train.shape)
'''
# 4-12
y_train[0:10]
# データの整形
# 説明変数のデータフレーム
X_train_df = pd.DataFrame(
    X_train, columns = ["sepal_len", "sepal_wid"])
# 応答変数のデータフレーム
y_train_df = pd.DataFrame({"species": y_train - 1})
# データフレームを結合
iris_train_df = pd.concat(
    [y_train_df, X_train_df], axis = 1)
'''
# 結果を出力
print(iris_train_df.head(3))
'''
# モデル化
# すべての変数を入れたモデル
logi_mod_full = smf.glm(
    "species ~ sepal_len + sepal_wid", data = iris_train_df,
    family = sm.families.Binomial()).fit()
# 長さのみ
logi_mod_len = smf.glm(
    "species ~ sepal_len", data = iris_train_df,
    family = sm.families.Binomial()).fit()
# 幅のみ
logi_mod_wid = smf.glm(
    "species ~ sepal_wid", data = iris_train_df,
    family = sm.families.Binomial()).fit()
# Nullモデル
logi_mod_null = smf.glm(
    "species ~ 1", data = iris_train_df,
    family = sm.families.Binomial()).fit()
'''
# AICの比較
print("full", logi_mod_full.aic.round(3))
print("len", logi_mod_len.aic.round(3))
print("wid", logi_mod_wid.aic.round(3))
print("null", logi_mod_null.aic.round(3))
logi_mod_len.summary().tables[1]
'''
# データの整形
X_test_df = pd.DataFrame(
    X_test, columns = ["sepal_len", "sepal_wid"])
# 当てはめと予測
logi_fit = logi_mod_len.fittedvalues.round(0)
logi_pred = logi_mod_len.predict(X_test_df).round(0)
# 正答数
true_train = sp.sum(logi_fit == (y_train - 1))
true_test = sp.sum(logi_pred == (y_test - 1))
# 的中率
result_train = true_train / len(y_train)
result_test = true_test / len(y_test)
'''
# 結果の出力
print("訓練データの的中率:", result_train)
print("テストデータの的中率:", result_test)
'''
# 4-13
# 標準化のための準備
scaler = StandardScaler()
scaler.fit(X_train)
# 標準化する
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
sp.std(X_train_scaled, axis = 0)
sp.std(X_test_scaled, axis = 0)
# 4-14
nnet = MLPClassifier(
    hidden_layer_sizes = (100,100),
    alpha = 0.07,
    max_iter = 10000,
    random_state = 0)
nnet.fit(X_train_scaled, y_train)
# 正答率
print("訓練データの的中率:", nnet.score(X_train_scaled, y_train))
print("テストデータの的中率:", nnet.score(X_test_scaled, y_test))

訓練データの的中率: 0.8933333333333333
テストデータの的中率: 0.68
